<a href="https://colab.research.google.com/github/utkar22/CSE508_Winter2023_A2_48/blob/main/IR_A2_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pickle 
import numpy as np
import pandas as pd
import os
import math

# **Jaccard Coefficient**


In [5]:
mod_path = '/content/drive/MyDrive/IR_Assignments/A2/modules'
ds_path = '/content/drive/MyDrive/IR_Assignments/A2/Processed_Dataset'

In [ ]:

doc_id_to_set = {}
idx = 1
for doc in sorted(os.listdir(ds_path)):
  mset = set()

  doc_id = int(doc[-4:])
  doc_path = ds_path + '/' + doc

  print(doc_id)

  with open(doc_path, 'r') as f:
    text = str(f.readlines()[0])

  tokens = text.split(' ')


  for token in tokens:
    mset.add(token)

  print(len(mset))
  print(len(tokens))
  doc_id_to_set[doc_id] = mset

print(len(doc_id_to_set))  
with open(mod_path+'/doc_id_to_mset.pkl', 'wb') as f:
  pickle.dump(doc_id_to_set, f)



In [ ]:
#calculating Jaccard coefficient of each query document wise
def calc_jacc(query_tokens, doc_id_to_set):
  res = []
  res_set = {}
  query_set = set()
  for token in query_tokens: #make the set of the query tokens
    query_set.add(token)

  for doc_id, mset in doc_id_to_set.items(): #calculate the intersection and union of each document set and query set
    un, inter = len(query_set.union(mset)), len(query_set.intersection(mset))
    if un == 0: #edge case when the doc or query do not have any words in them
      res_set[doc_id] = 0
      continue
    
    jacc_coeff = inter/un #calculating the Jaccard Coefficient
    res_set[doc_id] = jacc_coeff
  res_set = {k:v for k,v in sorted(res_set.items(), key = lambda item:item[1], reverse = True)} #sorting in descending order on the basis of the Jaccard Coefficient
  idx = 0
  for doc_id, jacc_coeff in res_set.items(): # make the list of the top 10 documents on the basis of the Jaccard coefficient
    res.append((doc_id, jacc_coeff))
    if idx == 9:
      break
    idx+=1

  return res




In [ ]:
#query processing and result printing
def jacc_coeff_queries():
  with open(mod_path+'/doc_id_to_mset.pkl', 'rb') as f:
    doc_id_to_set = pickle.load(f)
  q = int(input('Enter number of queries: '))
  while q>0:
    query = input('Enter query: ')

    query_tokens = query.split(' ')
    res = calc_jacc(query_tokens, doc_id_to_set)

    if res[0][1] == 0.0:
      print('No right document exists')
    else:
      print(res)
    q-=1

jacc_coeff_queries()

Enter number of queries: 3
Enter query: velocity temperature aerodynamics
[(407, 0.06896551724137931), (378, 0.05263157894736842), (549, 0.05263157894736842), (71, 0.05128205128205128), (31, 0.05), (269, 0.04878048780487805), (61, 0.047619047619047616), (154, 0.047619047619047616), (460, 0.0425531914893617), (485, 0.041666666666666664)]
Enter query: complete
[(684, 0.03225806451612903), (1038, 0.025), (1400, 0.022222222222222223), (372, 0.021739130434782608), (1078, 0.019230769230769232), (1135, 0.019230769230769232), (702, 0.018867924528301886), (319, 0.018518518518518517), (55, 0.017543859649122806), (1091, 0.017241379310344827)]
Enter query: entry
[(1348, 0.02702702702702703), (715, 0.022727272727272728), (1345, 0.018867924528301886), (944, 0.018518518518518517), (967, 0.01818181818181818), (275, 0.017543859649122806), (1394, 0.016666666666666666), (1346, 0.01639344262295082), (69, 0.015873015873015872), (982, 0.015873015873015872)]


# **TF IDF**

In [ ]:
vocabulary = set()

for doc in os.listdir(ds_path):
  with open(ds_path+f'/{doc}', 'r') as f:
    text = f.readlines()[0]
  tokens = text.split(' ')
  for token in tokens:
    vocabulary.add(token)
print(len(vocabulary))

8971


In [ ]:
idx = 0
word_to_index = {}
index_to_word = {}
for word in vocabulary:
  word_to_index[word] = idx
  index_to_word[idx] = word
  idx+=1
print(word_to_index)
print(index_to_word)
print(len(word_to_index))
print(len(index_to_word))

In [ ]:
with open(mod_path + '/vocabulary_set.pkl', 'wb') as f:
  pickle.dump(vocabulary, f)
with open(mod_path + '/word_to_index.pkl', 'wb') as f:
  pickle.dump(word_to_index, f)
with open(mod_path + '/index_to_word.pkl', 'wb') as f:
  pickle.dump(index_to_word, f)


In [12]:
with open(mod_path + '/vocabulary_set.pkl', 'rb') as f:
  vocabulary = pickle.load(f)
with open(mod_path + '/word_to_index.pkl', 'rb') as f:
  word_to_index = pickle.load(f)
with open(mod_path + '/index_to_word.pkl', 'rb') as f:
  index_to_word = pickle.load(f)

In [13]:
with open(mod_path + '/vocabulary_set.pkl', 'rb') as f:
  vocabulary = pickle.load(f)

In [ ]:
class Word:
  def __init__(self):
    self.binary_tf = {}
    self.count_tf = {}
    self.freq_tf = {}
    self.log_tf = {}
    self.double_norm_tf ={}
    self.df = 0
  

In [ ]:
def update_word(token, doc_id, total_words):
  ''' update the binary-tf, count-tf and 
  freq-tf weights of each word per doc'''
  if doc_id not in token.binary_tf:

    token.binary_tf[doc_id] = 1
    token.count_tf[doc_id] = 1

    token.freq_tf[doc_id] = float(token.count_tf[doc_id]/total_words)
    token.log_tf[doc_id] = math.log(1+token.count_tf[doc_id])
    
  else:
    token.count_tf[doc_id] = token.count_tf[doc_id]+1
    token.freq_tf[doc_id] = float(token.count_tf[doc_id]/total_words)

    token.log_tf[doc_id] = math.log(1+token.count_tf[doc_id])
  token.df = len(token.binary_tf)

def update_double_normal_tf(token, doc_id, max_count):
  ''' updates the double normalization tf value for a word'''
  token.double_norm_tf[doc_id] = 0.5 + 0.5*(token.count_tf[doc_id])/max_count


In [ ]:
lst = []
word_set = {}
index = 0
for doc in os.listdir(ds_path):
  local_dict = {}
  doc_idx = int(doc[-4:])
  lst.append(doc_idx)
  with open(ds_path + f'/{doc}', 'r') as f:
    text = f.readlines()[0]
  tokens = text.split(' ')
  print(tokens)
  for token in tokens:
    if token in local_dict:
      local_dict[token] += 1
    else:
      local_dict[token] = 1
    if token not in word_set:
      new_tok = Word()
      word_set[token] = new_tok
    update_word(word_set[token], doc_idx, len(tokens))

  sorted_dict = {k:v for k,v in sorted(local_dict.items(), key = lambda item:item[1], reverse = True)}
  maxi = sorted_dict[list(sorted_dict.keys())[0]]
  for token in tokens:
    update_double_normal_tf(word_set[token], doc_idx, maxi)
  

with open(mod_path + f'/word_set.pkl', 'wb') as f:
  pickle.dump(word_set, f)


In [ ]:
print(len(word_set))

8971


In [ ]:
with open(mod_path+ '/word_set.pkl', 'rb') as f:
  word_set = pickle.load(f)
with open(mod_path + '/word_to_index.pkl', 'rb') as f:
  word_to_index = pickle.load(f)
with open(mod_path + '/index_to_word.pkl', 'rb') as f:
  index_to_word = pickle.load(f)
print(len(word_set))
print(len(word_to_index))
print(len(index_to_word))


8971
8971
8971


In [ ]:
def make_binary_tf_idf_matrix(word_set, V, dir_len):
  binary_mat = np.zeros((dir_len+1, V))

  for token, word in word_set.items():
    map = word.binary_tf
    col = word_to_index[token]
    for row in map.keys():
      binary_mat[row][col] = math.log((1+dir_len)/(1+word.df))

  return binary_mat

binary_mat = make_binary_tf_idf_matrix(word_set, 8971, 1400)
print(np.count_nonzero(binary_mat==0))

12476867


In [ ]:
def make_count_tf_idf_matrix(word_set, V, dir_len):
  count_mat = np.zeros((dir_len+1, V))

  for token, word in word_set.items():
    map = word.count_tf
    col = word_to_index[token]
    for row in map.keys():
      count_mat[row][col] = map[row] * math.log((1+dir_len)/(1+word.df))

  return count_mat 

count_mat = make_count_tf_idf_matrix(word_set, 8971, 1400) 
print(np.count_nonzero(count_mat==0))

12476867


In [ ]:
def make_freq_tf_idf_matrix(word_set, V, dir_len):
  freq_mat = np.zeros((dir_len+1, V))

  for token, word in word_set.items():
    map = word.freq_tf
    col = word_to_index[token]
    for row in map.keys():
      freq_mat[row][col] = map[row] * math.log((1+dir_len)/(1+word.df))

  return freq_mat 
freq_mat = make_freq_tf_idf_matrix(word_set, 8971, 1400)
print(np.count_nonzero(freq_mat == 0))

12476867


In [ ]:
def make_log_tf_idf_matrix(word_set, V, dir_len):
  log_mat = np.zeros((dir_len+1, V))

  for token, word in word_set.items():
    map = word.log_tf
    col = word_to_index[token]
    for row in map.keys():
      log_mat[row][col] = map[row] * math.log((1+dir_len)/(1+word.df))

  return log_mat 
log_mat = make_log_tf_idf_matrix(word_set, 8971, 1400)
print(np.count_nonzero(log_mat == 0))
print(np.unique(log_mat))

12476867
[ 0.          0.49394171  0.5901114  ... 14.73825387 15.08606404
 16.59879738]


In [ ]:
def make_dnorm_tf_idf_matrix(word_set, V, dir_len):
  dnorm_mat = np.zeros((dir_len+1, V))

  for token, word in word_set.items():
    map = word.double_norm_tf
    col = word_to_index[token]
    for row in map.keys():
      dnorm_mat[row][col] = map[row] * math.log((1+dir_len)/(1+word.df))

  return dnorm_mat
dnorm_mat = make_dnorm_tf_idf_matrix(word_set, 8971, 1400)
print(np.count_nonzero(dnorm_mat == 0))
print(np.unique(dnorm_mat))

12476867
[0.         0.37505645 0.38175389 ... 6.08380905 6.14632926 6.55179437]


In [ ]:
with open(mod_path+'/dnorm_mat.pkl', 'wb') as f:
  pickle.dump(dnorm_mat, f)
with open(mod_path+'/log_mat.pkl', 'wb') as f:
  pickle.dump(log_mat, f)
with open(mod_path + '/freq_mat.pkl', 'wb') as f:
  pickle.dump(freq_mat, f)
with open(mod_path + '/count_mat.pkl', 'wb') as f:
  pickle.dump(count_mat, f)
with open(mod_path + '/binary_mat.pkl', 'wb') as f:
  pickle.dump(binary_mat, f)


In [8]:
with open(mod_path+'/dnorm_mat.pkl', 'rb') as f:
  dnorm_mat = pickle.load(f)
with open(mod_path+'/log_mat.pkl', 'rb') as f:
  log_mat = pickle.load(f)
with open(mod_path + '/freq_mat.pkl', 'rb') as f:
  freq_mat = pickle.load(f)
with open(mod_path + '/count_mat.pkl', 'rb') as f:
  count_mat = pickle.load(f)
with open(mod_path + '/binary_mat.pkl', 'rb') as f:
  binary_mat = pickle.load(f)

In [9]:
print(binary_mat.shape)

(1401, 8971)


In [1]:
import os
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
from typing import List
class preprocessor:
  def __init__(self):
    self.value = 1
  
  def read_file(self, path:str) -> str:
    '''reads the contents of a file and converts them into 
    a string of words separated by space.'''
    f = open(path, 'r')
    text = ''
    for line in f:
      for word in line.split(" "):
        text += '' + word + ' '
    f.close()
    return text
  
  def tokenize(self, text:str):
    ''' The function takes in a string and 
    converts it into a list of tokens using nltk'''

    tokens = word_tokenize(text)
    return tokens
  def tokens_to_text(self, tokens:List) -> str:
    ''' converts the tokens into space-separated string'''
    text = ''
    for token in tokens:

      text += '' + token + ' '
    return text

  def lowercase(self, tokens:List):
    ''' This function takes in the list of tokens
     and returns the lowercase version of each token without changing the index positions'''

    for i in range(len(tokens)):
      tokens[i] = tokens[i].lower()
    return tokens

  def remove_stopwords(self, tokens:List) -> List:
    ''' removes any stop words present as tokens in the list of tokens 
    present using nltk's corpus of stopwords'''
    stop_words = set(stopwords.words('english'))
    resultant_tokens = []
    for token in tokens:
      if token not in stop_words:
        resultant_tokens.append(token)
    return resultant_tokens

  def remove_puncts(self, tokens:List) -> List:
    ''' removes any punctuations present as tokens 
    and returns the resutlant list of tokens'''

    resultant_tokens = []
    text = self.tokens_to_text(tokens)
    text = text.translate(str.maketrans('', '', string.punctuation))

    return self.tokenize(text)
  
  

In [35]:
proc = preprocessor()
def preprocess_query(phrase:str)->List:
  phrase_tokens = proc.tokenize(phrase)
  phrase_tokens = proc.remove_puncts(phrase)
  phrase_tokens = proc.lowercase(phrase_tokens)
  phrase_tokens = proc.remove_stopwords(phrase_tokens)
  return phrase_tokens
def vectorize(query):
  tokens = preprocess_query(query)
  if len(tokens) == 0:
    return []
  query_map = {}
  total_len = len(tokens)

  for token in tokens:
    if token not in query_map:
      query_map[token] = 1
    else:
      query_map[token] += 1

  binary_vector = np.zeros((8971,1))
  count_vector = np.zeros((8971,1))
  freq_vector = np.zeros((8971,1))
  log_vector = np.zeros((8971,1))
  dnorm_vector = np.zeros((8971,1))

  sorted_dict = {k:v for k,v in sorted(query_map.items(), key = lambda item:item[1], reverse = True)}
  maxi = sorted_dict[list(sorted_dict.keys())[0]]

  for word in query_map.keys():
    if word in word_to_index:
      binary_vector[word_to_index[word]][0] = 1
      count_vector[word_to_index[word]][0] = query_map[word]
      freq_vector[word_to_index[word]][0] = query_map[word]/total_len
      log_vector[word_to_index[word]][0] = math.log(1+query_map[word])
      dnorm_vector[word_to_index[word]][0] = 0.5 + 0.5*(query_map[word])/maxi

  res = []
  res.append(binary_vector)
  res.append(count_vector)
  res.append(freq_vector)
  res.append(log_vector)
  res.append(dnorm_vector)

  return res  

def calc_res(matrix, vector): 
  ''' returns the top 10 documents on the basis of the cosine similarity of the doc and the query tf vector'''
  res_dict = {}
  for row in range(1, matrix.shape[0]):
    res_dict[row] = np.dot(matrix[row], vector)
  sorted_dict = {k:v for k,v in sorted(res_dict.items(), key = lambda item:item[1], reverse = True)}
  idx = 0
  res_list = []
  for doc_id in sorted_dict.keys():
    res_list.append(doc_id)
    if idx == 9:
      break
    idx+=1
  return res_list
    

In [10]:
vocab_matrix = []
vocab_matrix.append(binary_mat)
vocab_matrix.append(count_mat)
vocab_matrix.append(freq_mat)
vocab_matrix.append(log_mat)
vocab_matrix.append(dnorm_mat)
print(len(vocab_matrix))

5


In [37]:
def run_tf_idf():
  q = int(input('Enter the number of queries: '))
  while q>0:
    query = input('Enter the query: ')
    vectors = vectorize(query)
    if len(vectors) == 0:
        print('No relevant documents found. All dot products were 0')
        break
    type_of_tf = ['Binary weights', 'Count as weight', 'Frequency as weight', 'Log normalised weight', 'Double normalization weight']
    for i in range(5):
      print(f'Type of Tf: {type_of_tf[i]}')
      res_list = calc_res(vocab_matrix[i], vectors[i])
      if len(res_list) == 0:
        print('No relevant documents found. All dot products were 0')
      else:
        print(res_list)
    q-=1
run_tf_idf()

    

Enter the number of queries: 3
Enter the query: velocity is it
Type of Tf: Binary weights
[1365, 857, 962, 1138, 263, 437, 452, 458, 731, 787]
Type of Tf: Count as weight
[731, 1365, 1082, 1047, 455, 528, 548, 1218, 857, 962]
Type of Tf: Frequency as weight
[1365, 1138, 437, 731, 528, 429, 612, 159, 615, 380]
Type of Tf: Log normalised weight
[731, 1365, 1047, 1082, 857, 962, 1138, 263, 437, 452]
Type of Tf: Double normalization weight
[1365, 1138, 437, 731, 1047, 263, 1056, 857, 962, 611]
Enter the query: analytics and analysis
Type of Tf: Binary weights
[962, 1136, 673, 1040, 1092, 1330, 8, 739, 857, 890]
Type of Tf: Count as weight
[259, 867, 131, 304, 476, 1047, 1092, 1365, 1040, 249]
Type of Tf: Frequency as weight
[259, 867, 437, 478, 1365, 1298, 855, 249, 476, 868]
Type of Tf: Log normalised weight
[259, 1092, 867, 1040, 131, 304, 476, 1047, 1136, 1365]
Type of Tf: Double normalization weight
[1136, 1092, 1365, 259, 437, 867, 1040, 962, 1330, 673]
Enter the query: isi=
No releva